In [1]:
from IPython.core.display import display, HTML, Javascript
from string import Template
import json, os
import pandas as pd
import numpy as np

In [2]:
notebook_path = os.path.abspath("Labor_Stats2.ipynb")
labor_csv = os.path.join(os.path.dirname(notebook_path), "data/", "nychist_nonfarm.csv")
labor = pd.read_csv(labor_csv)
del labor['AREA']
del labor['SERIESCODE']
del labor['AREANAME']
del labor['INDUSTRY_TITLE']
del labor['ANNUAL']

In [3]:
labor_filtered = labor.loc[(labor['YEAR'] <= 2020) & (labor['YEAR'] >= 2011)]
labor_filtered.to_csv('labor_forObservable.csv')
labor_filtered.head()

,YEAR,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC
1,2020,4638.9,4675.0,4631.4,3748.5,3795.9,3860.0,3909.5,3970.0,4040.2,4102.4,4129.6,4100.2
2,2019,4541.5,4590.8,4620.8,4639.5,4649.0,4661.9,4627.0,4612.7,4647.6,4700.1,4753.9,4756.7
3,2018,4433.2,4490.1,4518.1,4528.5,4545.5,4565.0,4532.9,4512.7,4546.3,4622.7,4665.5,4674.0
4,2017,4354.7,4402.1,4421.3,4432.3,4457.8,4466.7,4456.5,4433.3,4458.2,4522.9,4561.1,4578.6
5,2016,4274.7,4314.1,4340.2,4369.5,4362.3,4368.8,4373.7,4353.8,4371.9,4427.1,4463.5,4481.3


In [4]:
data = labor_filtered.to_dict(orient='records')
#print(data)

In [5]:
%%javascript
require.config({
    paths: {
        d3: "https://d3js.org/d3.v6.min"
     }
});

require(["d3"], function(d3) {
    console.log(d3);
    window.d3 = d3;
});

<IPython.core.display.Javascript object>

In [6]:
html_template = Template('''
<style> $css_text </style>
<div id="graph-div"></div>
<script> $js_text </script>
''')

In [7]:
css_text = ''''''

In [8]:
js_text_template = Template('''
  var margin = {top: 10, right: 50, bottom: 20, left: 40};
  var visWidth = 900 - margin.left - margin.right;
  var visHeight = 500 - margin.top - margin.bottom;
  
  var svg = d3.select("#$graphdiv")
    .append('svg')
    .attr('width', visWidth + margin.left + margin.right)
    .attr('height', visHeight + margin.top + margin.bottom);  
  
  var g = svg.append('g')
    .attr('transform', `translate(${margin.left}, ${margin.top})`);
  
  var months = ["JAN", "FEB", "MAR", "APR", "MAY", "JUN", "JUL", "AUG", "SEP", "OCT", "NOV", "DEC"];
  
  data1 = $python_data;
  
  var data = data1.map(d => {
    var o = {};
    o.year = parseInt(d.YEAR);
    o.data = Object.entries(d).slice(1);
    o.data = o.data.map(d => {
     var o1 = {};
     o1.Month = d[0];
     o1.count = parseFloat(d[1]);
     return o1;
    })
    return o;
    }
  ).sort((a, b) => d3.ascending(a.year, b.year));
  
  var minCount = [];
  var maxCount = [];
  for (var i = 0; i < data.length; i++) {
    minCount.push(d3.min(data[i].data, d => d.count));
    maxCount.push(d3.max(data[i].data, d => d.count));
  }
  var minMaxCount = [d3.min(minCount), d3.max(maxCount)];
  
  var x = d3.scalePoint()
    .domain(months)
    .range([0, visWidth]);
  
  var y = d3.scaleLinear()
    .domain([minMaxCount[0] - 200, minMaxCount[1] + 100]).nice()
    .range([visHeight, 0]); 
  
  var xAxis = d3.axisBottom(x)
  var yAxis = d3.axisLeft(y);  
  
  g.append('g')
    .attr('transform', `translate(0,${visHeight})`)
    .call(xAxis);  
  
  g.append('g')
      .call(yAxis)
      .call(g => g.selectAll('.domain').remove())
    .append('text')
      .attr('text-anchor', 'start')
      .attr('dominant-baseline', 'middle')
      .attr('fill', 'black')
      .attr('x', 5)
      .style('font-size', 14)
      .text('NYC Non-Farm Labor Participation (in thousands of people)');    
  
  var line = d3.line()
      .x(d => x(d.Month))
      .y(d => y(d.count));  
  
  var series = g.append('g')
    .selectAll('g')
    .data(data)
    .join('g')
      .attr('stroke', d => d.year == 2020 ? 'green' : 'lightgray')
    .append('path')
      .datum(d => d.data)
      .attr('fill', 'none')
      .attr('stroke-width', 2)
      .attr('d', line);   
  
  g.selectAll('.year-label')
    .data(data)
    .join('text')
      .attr('class', 'year-label')
      .attr('x', d => x(d.data[11].Month) + 2)
      .attr('y', d => y(d.data[11].count))
      .attr('font-size', 10)
      .attr('font-family', 'sans-serif')
      .attr('dominant-baseline', 'middle')
      .text(d => d.year);  
''')

In [9]:
html_template = Template('''
<style> $css_text </style>
<div id="graph-div"></div>
<script> $js_text </script>
''')
js_text = js_text_template.safe_substitute(python_data=json.dumps(data),
                                       graphdiv='graph-div')
HTML(html_template.safe_substitute(css_text=css_text, js_text=js_text))